<a href="https://colab.research.google.com/github/anantterkar/Mini-Project-I/blob/main/BART_base_and_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing all necessary libraries

In [ ]:
!pip install transformers
!pip install rouge_score


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 104.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=cbb19597459ed8c0b400135b4c0494482fe2499cf94d85a2a6a9fdd71a4b274a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import pandas as pd
from transformers import *
#from blurr.data.all import *
#from blurr.modeling.all import *

import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation
from nltk.stem.wordnet import WordNetLemmatizer
from tqdm import tqdm
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
import time
import datetime
from scipy import sparse
import numpy as np

from sklearn.model_selection import train_test_split

/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `src/transformers/generation_flax_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import FlaxGenerationMixin` instead.
  warnings.warn(


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

#Cleaning Data

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Mini-project/dataset.csv')
training_data = df.sample(frac=0.8, random_state=25)
testing_data = df.drop(training_data.index)


In [ ]:
#removing dates and URL from Decisions
import re

def remove_date(df):
    pattern = r'\b\d{2}/\d{2}/\d{4}\b'  # Regular expression pattern for DD-MM-YYYY format dates
    df = df.applymap(lambda x: re.sub(pattern, '', str(x)))
    return df

urls = r'\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))'

def remove_url(df):
  df = df.applymap(lambda x: re.sub(urls, '', str(x)))
  df = df.applymap(lambda x: re.sub('URL: ', '', str(x)))
  return df

foot = 'LIIofIndia: Copyright Policy | Disclaimers | Privacy Policy | Feedback'

def remove_foot(df):
  df = df.applymap(lambda x: re.sub(foot, '', str(x)))
  return df

training_data = remove_date(training_data)
testing_data = remove_date(testing_data)

training_data = remove_url(training_data)
testing_data = remove_url(testing_data)

training_data = remove_foot(training_data)
testing_data = remove_foot(testing_data)

#Model and Tokenizer

In [ ]:
import torch
import numpy as np


from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)

from tabulate import tabulate
import nltk
from datetime import datetime

In [ ]:
model_name = 'facebook/bart-large-cnn'
model_bartlarge = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer_bartlarge = AutoTokenizer.from_pretrained(model_name)
''' # tokenization
encoder_max_length = 256  # demo
decoder_max_length = 64 '''

In [ ]:
model_bart_base = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
tokenizerbb = BartTokenizer.from_pretrained('facebook/bart-base')

model_bart_large = BartForConditionalGeneration.from_pretrained('facebook/bart-large')
tokenizerbl = BartTokenizer.from_pretrained('facebook/bart-large')

In [ ]:
model_name = 'Udit191/autotrain-summarization-bart-50017120109'
ourmodel = AutoModelForSeq2SeqLM.from_pretrained(model_name)
ourtokenizer = AutoTokenizer.from_pretrained(model_name)

#Abbreviations Extracting

In [ ]:
#extracting all abbreviations from decisions essentially
import pandas as pd
import re
import os
import nltk
from nltk.stem import SnowballStemmer
os.chdir('D:\sem6\mini_project')
nltk.download('words')
path = './training_data.csv'
# Load Excel data into a Pandas DataFrame
df = pd.read_csv(path)

# Specify the column containing abbreviations
abbrev_col = 'Decisions'

# Create an empty list to store extracted abbreviations
abbrev_list = []

# Loop through each cell in the specified column and extract abbreviations using regex
for cell in df[abbrev_col]:
    abbrev_match = re.findall(r'\b[a-zA-Z]+\.', str(cell))
    abbrev_list += abbrev_match
abbrev_list = list(set(abbrev_list))
abbrev_list = [match[:-1] for match in abbrev_list]
english_words = set(nltk.corpus.words.words())
nltk.download('snowball_data')
stemmer = SnowballStemmer('english')
stemmed_words = [stemmer.stem(word) for word in abbrev_list]
filtered_words = [word for word in stemmed_words if word not in english_words]
print(len(filtered_words))


In [ ]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet

# Example word to get forms for
word = 'run'

# Get the synsets for the word
synsets = wordnet.synsets(word)

# Get the lemmas for each synset and their inflected forms
inflected_forms = []
for synset in synsets:
    for lemma in synset.lemmas():
        inflected_forms.extend(lemma.derivationally_related_forms())

# Get the unique inflected forms and print them
unique_inflected_forms = set(inflected_forms)
print(unique_inflected_forms)


[nltk_data] Downloading package wordnet to /root/nltk_data...


{Lemma('hunt.n.08.hunting'), Lemma('operation.n.08.functioning'), Lemma('disentangler.n.01.unraveller'), Lemma('plier.n.01.plier'), Lemma('racing.n.01.racing'), Lemma('range.n.06.range'), Lemma('racer.n.02.racer'), Lemma('fugitive.n.01.runaway'), Lemma('persistent.a.03.persistent'), Lemma('course.n.03.course'), Lemma('thaw.n.01.melting'), Lemma('test.v.06.test'), Lemma('consort.n.01.consort'), Lemma('executive.a.01.executive'), Lemma('running.s.06.operative'), Lemma('operant.s.01.operant'), Lemma('operation.n.11.operation'), Lemma('hunt.n.08.hunt'), Lemma('discharge.v.02.discharge'), Lemma('getaway.n.02.lam'), Lemma('meltable.s.01.meltable'), Lemma('inclination.n.05.inclination'), Lemma('campaign.v.01.campaign'), Lemma('run.v.24.run'), Lemma('black_market.n.01.black_market'), Lemma('fluid.s.02.runny'), Lemma('runner.n.02.runner'), Lemma('hunter.n.01.hunter'), Lemma('run.n.07.running'), Lemma('thaw.n.01.melt'), Lemma('flow.n.03.flow'), Lemma('operator.n.03.operator'), Lemma('race.n.02.r

In [ ]:
from nltk.stem import SnowballStemmer
import nltk
nltk.download('words')
path = '/content/drive/MyDrive/Mini-project/training_data.csv'

# Load Excel data into a Pandas DataFrame
df = pd.read_csv(path)

# Specify the column containing abbreviations
abbrev_col = 'Decisions'

# Create an empty list to store extracted abbreviations
abbrev_list = []

# Loop through each cell in the specified column and extract abbreviations using regex
for cell in df[abbrev_col]:
    abbrev_match = re.findall(r'\b[a-zA-Z]{5}\.\b', str(cell))
    abbrev_list += abbrev_match
abbrev_list = list(set(abbrev_list))
abbrev_list = [match[:-1] for match in abbrev_list]
english_words = set(nltk.corpus.words.words())
nltk.download('snowball_data')
stemmer = SnowballStemmer('english')
stemmed_words = [stemmer.stem(word) for word in abbrev_list]
filtered_words = [word for word in stemmed_words if word not in english_words]
print((filtered_words))


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


['laps', 'issu', 'pepsu', 'sens', 'asid', 'hous', 'entri', 'hindu', 'mahto', 'acr', 'leas', 'alon', 'hous', 'sabha', 'appli', 'marla', 'guid', 'forc', 'bihar', 'yadav', 'caus', 'fals', 'valu', 'titl', 'india', 'parti', 'vagu', 'versa', 'heard', 'basi', 'sinc', 'judg', 'faddi', 'aros', 'dutta', 'larg', 'tinu', 'gupta', 'delhi', 'abov', 'aliv', 'leav', 'katni', 'desai', 'gulab', 'fasli', 'bidi', 'poona', 'varma', 'naidu', 'aris']


[nltk_data] Downloading package snowball_data to /root/nltk_data...
[nltk_data]   Package snowball_data is already up-to-date!


# BART-LARGE-CNN

---


In [ ]:
training_data.columns


Index(['Decisions', 'Headnote'], dtype='object')

In [ ]:
len(training_data['Headnote'][0])

403

In [ ]:
# Set the maximum sequence length for the model
max_length = model_bartlarge.config.max_position_embeddings


# Function to generate summary for a given input text
def generate_summary_bartlarge(text):
    input_ids = tokenizer_bartlarge.encode(text, max_length=max_length, truncation=True, return_tensors='pt')
    summary_ids = model_bartlarge.generate(input_ids, num_beams=4, max_length=max_length, early_stopping=True)
    summary = tokenizer_bartlarge.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [ ]:
text = training_data['Decisions'][0]

In [ ]:
max_length

1024

In [ ]:
summary_text = generate_summary_bartlarge(text)

In [ ]:
summary_bartlarge = summary_text

In [ ]:
print(len(summary_text))
print(len(text))

432
20618


In [ ]:
input_text = text

# Split the input text into smaller segments
segment_length = 512
segments = [input_text[i:i+segment_length] for i in range(0, len(input_text), segment_length)]

In [ ]:
len(segments)

41

In [ ]:
# Generate summaries for each segment
summaries = []
for segment in segments:
    summary = generate_summary_bartlarge(segment)
    summaries.append(summary)

# Combine the summaries
final_summary = " ".join(summaries)
print("Final Summary:", final_summary)

In [ ]:
final_summary

'HIDAYATULLAH, M.DAYAL, RAGHUBARCITATION: 1964 AIR 1263 1964 SCR (6) 172CITATOR INFO :R 1977 SC2252 (4)ACT:Criminal Trial-Penal Code-Conviction under s. 304 Part II. the series of acts which result in the offence. In the case of a person assaulted by many accused, the criminal act is the offence which finally results. The achievement of that criminal act may be the result of the action of several persons.Knowledge in s. 304 Part 11 is the knowledge of likelihood of death. he likely consequence of the criminal act, there is no reason why s. 34 should not be read with the second part of s. 304 to make each liable individually.lbra Akanda v. Emperor, I.L.R. [1942] 2 Cal. 405 and Saidu Khan v. State, I,L. R [1952] 1 All, 639, approved.Ramnath v Emperor, A.I.R 1943 All. 271, Shahibzada V. The Crown A.R 1950 Peshawar 24, Debi Chand Haldar v Emperor,. 41 C.W.N. 570 and Barendra Kumar Ghosh v Emperor. (1925) I. l.R 52 Cal. 197. Appeal by special leave from the judgment and order dated March 5,

In [ ]:
# Set the maximum sequence length for the model
max_length = model.config.max_position_embeddings

# Function to generate summary for a given input text
def generate_summary(text):
    input_ids = tokenizer.encode(text, max_length=max_length, truncation=True, return_tensors='pt')
    summary_ids = model.generate(input_ids, num_beams=4, max_length=100, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Generate summaries for all rows and store in a new column 'summary'
training_data['summary'] = training_data['Decisions'].apply(generate_summary)

# Print the DataFrame with the added 'summary' column
print(training_data)

In [ ]:
training_data.to_csv('training_data.csv', index=False)

#Correcting Grammar


In [ ]:
!pip install language-tool-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from language_tool_python import LanguageTool

def correct_grammar(text):
    tool = LanguageTool('en-US')  # Specify the language you want to check

    # Check grammar and get suggestions
    matches = tool.check(text)

    # Correct grammar using suggestions
    corrected_text = LanguageTool.correct(text, matches)

    return corrected_text

# Example usage
concatenated_summary = final_summary

corrected_summary = correct_grammar(concatenated_summary)
print(corrected_summary)


AttributeError: ignored

# BART-BASE

In [ ]:
# Set the maximum sequence length for the model
max_length = model_bart_base.config.max_position_embeddings


# Function to generate summary for a given input text
def generate_summary_bartbase(text):
    input_ids = tokenizerbb.encode(text, max_length=max_length, truncation=True, return_tensors='pt')
    summary_ids = model_bart_base.generate(input_ids, num_beams=4, max_length=max_length, early_stopping=True)
    summary = tokenizerbb.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [ ]:
summary_bartbase = generate_summary_bartbase(text)

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.28.1"
}



#

#Our Model

In [ ]:
# Set the maximum sequence length for the model
max_length = model_bart_base.config.max_position_embeddings


# Function to generate summary for a given input text
def generate_summary_ourmodel(text):
    input_ids = ourtokenizer.encode(text, max_length=max_length, truncation=True, return_tensors='pt')
    summary_ids = ourmodel.generate(input_ids, num_beams=4, max_length=max_length, early_stopping=True)
    summary = ourtokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [ ]:
summary_ourmodel = generate_summary_ourmodel(text)

#ROUGE SCORES

In [ ]:
from rouge_score import rouge_scorer

In [ ]:
reference_summary = summary_bartbase


scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(reference_summary, text)

rouge1_bartbase = scores['rouge1'].fmeasure
rouge2_bartbase = scores['rouge2'].fmeasure
rougeL_bartbase = scores['rougeL'].fmeasure
print('Bart-base:')
print("ROUGE-1 Score:", rouge1_bartbase)
print("ROUGE-2 Score:", rouge2_bartbase)
print("ROUGE-L Score:", rougeL_bartbase)

Bart-base:
ROUGE-1 Score: 0.31163620151582705
ROUGE-2 Score: 0.26806422836752897
ROUGE-L Score: 0.23539901917075348


In [ ]:
reference_summary = summary_bartlarge
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(reference_summary, text)

rouge1_bartlarge = scores['rouge1'].fmeasure
rouge2_bartlarge = scores['rouge2'].fmeasure
rougeL_bartlarge = scores['rougeL'].fmeasure
print('Bart-base:')
print("ROUGE-1 Score:", rouge1_bartlarge)
print("ROUGE-2 Score:", rouge2_bartlarge)
print("ROUGE-L Score:", rougeL_bartlarge)

Bart-base:
ROUGE-1 Score: 0.03829006031995804
ROUGE-2 Score: 0.0372605615324062
ROUGE-L Score: 0.03776553894571204


In [ ]:
reference_summary = final_summary
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(reference_summary, text)

rouge1 = scores['rouge1'].fmeasure
rouge2 = scores['rouge2'].fmeasure
rougeL = scores['rougeL'].fmeasure
print('Bart_large_cnn_divided:')
print("ROUGE-1 Score:", rouge1)
print("ROUGE-2 Score:", rouge2)
print("ROUGE-L Score:", rougeL)

Bart_large_cnn_divided:
ROUGE-1 Score: 0.7861048568671598
ROUGE-2 Score: 0.741956241956242
ROUGE-L Score: 0.7137343197169507


In [ ]:
final_summary

'HIDAYATULLAH, M.DAYAL, RAGHUBARCITATION: 1964 AIR 1263 1964 SCR (6) 172CITATOR INFO :R 1977 SC2252 (4)ACT:Criminal Trial-Penal Code-Conviction under s. 304 Part II. the series of acts which result in the offence. In the case of a person assaulted by many accused, the criminal act is the offence which finally results. The achievement of that criminal act may be the result of the action of several persons.Knowledge in s. 304 Part 11 is the knowledge of likelihood of death. he likely consequence of the criminal act, there is no reason why s. 34 should not be read with the second part of s. 304 to make each liable individually.lbra Akanda v. Emperor, I.L.R. [1942] 2 Cal. 405 and Saidu Khan v. State, I,L. R [1952] 1 All, 639, approved.Ramnath v Emperor, A.I.R 1943 All. 271, Shahibzada V. The Crown A.R 1950 Peshawar 24, Debi Chand Haldar v Emperor,. 41 C.W.N. 570 and Barendra Kumar Ghosh v Emperor. (1925) I. l.R 52 Cal. 197. Appeal by special leave from the judgment and order dated March 5,

In [ ]:
gold_std = training_data['Headnote'][0]

# ROUGE SCORES against gold std

In [ ]:
reference_summary = summary_bartlarge
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(reference_summary, gold_std)

rouge1_bartbase = scores['rouge1'].fmeasure
rouge2_bartbase = scores['rouge2'].fmeasure
rougeL_bartbase = scores['rougeL'].fmeasure
print('Bart-base:')
print("ROUGE-1 Score:", rouge1_bartbase)
print("ROUGE-2 Score:", rouge2_bartbase)
print("ROUGE-L Score:", rougeL_bartbase)

Bart-base:
ROUGE-1 Score: 0.54421768707483
ROUGE-2 Score: 0.33103448275862074
ROUGE-L Score: 0.380952380952381


In [ ]:
reference_summary = summary_bartbase
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(reference_summary, gold_std)

rouge1_bartbase = scores['rouge1'].fmeasure
rouge2_bartbase = scores['rouge2'].fmeasure
rougeL_bartbase = scores['rougeL'].fmeasure
print('Bart-base:')
print("ROUGE-1 Score:", rouge1_bartbase)
print("ROUGE-2 Score:", rouge2_bartbase)
print("ROUGE-L Score:", rougeL_bartbase)

Bart-base:
ROUGE-1 Score: 0.16585365853658537
ROUGE-2 Score: 0.09779951100244498
ROUGE-L Score: 0.10487804878048779


In [ ]:
reference_summary = final_summary
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(reference_summary, gold_std)

rouge1_bartbase = scores['rouge1'].fmeasure
rouge2_bartbase = scores['rouge2'].fmeasure
rougeL_bartbase = scores['rougeL'].fmeasure
print('Bart-base:')
print("ROUGE-1 Score:", rouge1_bartbase)
print("ROUGE-2 Score:", rouge2_bartbase)
print("ROUGE-L Score:", rougeL_bartbase)

Bart-base:
ROUGE-1 Score: 0.054858934169279
ROUGE-2 Score: 0.038431372549019606
ROUGE-L Score: 0.04467084639498432


In [ ]:
reference_summary = """The ban on transfers which may affect the ceiling law is more severe under the Bombay Tenancy and Agricultural Lands Act, 1948. Sec. 63 directs that no sale of land shall be valid in favour of a person who will, after such sale, hold land exceeding two-thirds of the ceiling area deter- mined under the Maharashtra Agricultural Lands (Ceiling on Holdings) Act, 1961. The fact that on the death of A in 1962 his earlier holdings were inherited by his heirs and the respective holdings, therefore, came below the ceiling area, is immaterial because the disputed land was purchased by A himself in 1960. It has to be remembered that, as has been held by the Revenue Tribunal, the other lands exclusively belonged to A and exceeded the ceiling area. The sale on 3.6.1960 must, therefore, be held to be illegal and inopera- tive. [241G-H; 242A]

CIVIL APPELLATE JURISDICTION: Civil Appeal No. 950 (N) of 1973.

From the Judgment and Order dated 2.8.1972 of the Bombay High Court in Special Civil Application No. 2826 of 1969.

Pinaki Misra, P.H. Parekh and Ms. Sunita Sharma for the Appellants.

V .N. Ganpule and V.D. Khanna for the Respondents.

The Judgment of the Court was delivered by SHARMA, J. The subject matter of this appeal is 13.30 acres of land in Sholapur District, within the State of Maharashtra. The appellants are the heirs of one Mugaji Laxman Padule, who was the tenant of the land for about 3 decades before the Bombay Tenancy and Agricultural Land Act, 1948 (hereinafter referred to as the Act) was 240 enacted. Under the provisions of the Act, Mugaji was enti- tled to purchase the land on satisfaction of certain condi- tions. Admittedly he did not satisfy these conditions and said so before the authorities concerned. The landlords who are now represented by the respondents, were claiming pos- session of the area under the Act. Mugaji, subsequently, made a claim to the Land on another basis. On his death in 1962, his heirs the appellants were substituted. The matter was considered by several authorities under the Act, who ultimately rejected the appellants' case. The appellants, thereafter moved the Bombay High Court by an application under Article 227 of the Constitution of India, which was rejected by the impugned judgment.

2. The procedure for the tenant to purchase the land is laid down in Sec. 32G of the Act. It enjoins the Agricultur- al Land Tribunal constituted under Sec. 67 to publish a public notice calling upon the tenants, the landlords and any other interested person to appear before it on a speci- fied date. The Tribunal is also required to issue individual notices to the landlords and the tenants, and thereafter to decide the competing cases. Sec. 32A limits the right of a tenant holding other Lands to such area only which will raise his holding to the extent of the ceiling area. Admit- tedly Mugaji was already possessed of lands beyond the ceiling area and he, therefore, did not claim to have pur- chased the land in accordance with the provisions of the Act. In a situation where a tenant is not able to success- fully claim the land, it has to be disposed of in the manner provided in Sec. 32P, which states that the former tenant would be summarily evicted and the land would be surrendered to the landlord. In the present case the land in question, thus, went to the landlords. According to the case of the appellants, on a partition in the family of the landlords the disputed land was allotted to the share of the respond- ents 2 to 4 and Mugaji purchased the same for a sum of Rs.

3,000 from them on 3.6.1960. The appellants alleged that by this date, i.e., 3.6.1960 the land held by Mugaji was within the ceiling area following a partition between him and his sons on 13.10.1959.

3. The respondents moved the authorities under the Act in 1963 for recognising their claim. They did not implead the appellants and suppressed the fact of the sale on 3.6.1960 in favour of Mugaji. The Agricultural Lands Tribu- nal and Additional Mamlatdar relying on the enquiry under Sec. 32G, wherein the right of Mugaji as a tenant was nega- tived, upheld the claim of the present respondents by his order dated 28.4.1963. When the appellants learnt about it, they filed an appeal before the Collector. They also chal- lenged the earlier order 241 passed against Mugaji under Sec. 32G. The Collector remanded the matter on 25.9.1963. The Additional Mamlatdar by his order dated 8.2. 1964 observed that since the tenant had purchased the suit land from the landlords, the proceeding was fit to be dropped and it was appropriate to deal with the case under Sec. 84C of the Act. Sec. 84C states that in respect of a transfer of any land made after 1955 if the Mamlatdar has reason to believe that the transfer was in- valid on account of any of the provisions of the Act, he would issue notice and hold an enquiry and decide whether the transfer is invalid or not. In 1965 a further order was passed in the case wherein the Agricultural Lands Tribunal held the purchase by Mugaji on 3.6.1960 as lawful and upheld the claim of the appellants. The order was upheld in appeal, and the respondent No. 3 filed a revision application before the Revenue Tribunal. It was contended on behalf of the present appellants that after the partition between Mugaji and his sons in 1959 the area held by him came below the ceiling level and he was, thus. entitled to purchase the land on 3.6. 1960. The Maharashtra Revenue Tribunal held that the land owned by Mugaji did not belong to the joint family and his sons had no share therein, and the alleged partition, therefore, could not be accepted or recognised.

The result is that even in 1960, Mugaji was possessed of land beyond the ceiling area and he was not entitled to purchase further land from the respondents 2 to 4. Thus, having lost the case, the appellants moved the Bombay High Court, and their application was rejected by a short judg- ment passed on 2.8. 1972 which is under challenge in this appeal by Special Leave.

4. The learned counsel for appellants contended that the High Court was in error in assuming that the claim of the appellants was based on the right of Mugaji under Sec. 32G of the Act in the capacity of a tenant; and also in relying on Sec. 10 of the Maharashtra Agricultural Lands (Ceiling on Holdings) Act, 1961. The learned counsel appears to be right but for this reason the appellants can not succeed. The ban on transfers which may affect the ceiling law is more severe under the Bombay Tenancy and Agricultural Lands Act, 1948.

Sec. 63 directs that no sale of land shall be valid in favour of a person who will after such sale hold land ex- ceeding two-thirds of the ceiling area determined under the Maharashtra Agricultural lands (Ceiling on Holdings) Act, 1961. The fact that on the death of Mugaji in 1962 his earlier holdings were inherited by his heirs and the respec- tive holdings, therefore, came below the ceiling area is immaterial, because the disputed land was purchased by Mugaji himself in 1960. It has to be remembered that, as has been held by the Revenue Tribunal, the other 242 lands exclusively belonged to Mugaji and exceeded the ceil- ing area. The sale on 3.6.1960 must, therefore, be held to be illegal and inoperative. Consequently, the appellants must lose although for slightly different reasons than those given by the High Court. The appeal is accordingly dismissed but in the circumstances without costs.

G.N. Appeal dis- missed."""
generated_summary = """The appellants are the heirs of one Mugaji Laxman Padule, who was the tenant of the land for about 3 decades before the Bombay Tenancy and Agricultural Land Act, 1948 (hereinafter referred to as the Act) was enacted. On his death in 1962, the land was purchased by him for Rs. 3.6,000 from his son-in-law for the sum of Rs. 2.3,000 and the land in question was allotted to him and his sons. The respondents, who are represented by the respondents, were the tenants of the said land and were entitled to a share of the same under Sec. 32G of the Act."""

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(reference_summary, final_summary)

rouge1_score = scores['rouge1'].fmeasure
rouge2_score = scores['rouge2'].fmeasure
rougeL_score = scores['rougeL'].fmeasure

print("ROUGE-1 Score:", rouge1_score)
print("ROUGE-2 Score:", rouge2_score)
print("ROUGE-L Score:", rougeL_score)

ROUGE-1 Score: 0.39234449760765555
ROUGE-2 Score: 0.31497005988023946
ROUGE-L Score: 0.29306220095693775
